In [3]:
pip install pandas openpyxl requests pillow

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [1]:
pip install --upgrade pandas openpyxl

Defaulting to user installation because normal site-packages is not writeable
  Using cached pandas-2.2.2-cp311-cp311-win_amd64.whl.metadata (19 kB)
   ---------------------------------------- 0.0/11.6 MB ? eta -:--:--
   ---------------------------------------- 0.0/11.6 MB ? eta -:--:--
   ---------------------------------------- 0.1/11.6 MB 1.1 MB/s eta 0:00:11
    --------------------------------------- 0.2/11.6 MB 2.0 MB/s eta 0:00:06
   - -------------------------------------- 0.4/11.6 MB 2.4 MB/s eta 0:00:05
   - -------------------------------------- 0.6/11.6 MB 3.2 MB/s eta 0:00:04
   -- ------------------------------------- 0.7/11.6 MB 3.0 MB/s eta 0:00:04
   -- ------------------------------------- 0.7/11.6 MB 3.0 MB/s eta 0:00:04
   ---- ----------------------------------- 1.2/11.6 MB 3.7 MB/s eta 0:00:03
   ----- ---------------------------------- 1.5/11.6 MB 3.7 MB/s eta 0:00:03
   ------ --------------------------------- 1.8/11.6 MB 4.2 MB/s eta 0:00:03
   ------ --------

In [22]:
import os
import tkinter as tk
from tkinter import ttk, messagebox, filedialog
from datetime import datetime
import requests
import pandas as pd
import json
from PIL import Image, ImageTk  # PIL kütüphanesi

# OpenAI API ayarları
API_KEY = 'Y'
API_URL = "https://api.openai.com/v1/chat/completions"

# Twitter API ayarları
BEARER_TOKEN = ''
search_url = "https://api.twitter.com/2/tweets/search/recent"


def create_headers(bearer_token):
    headers = {"Authorization": f"Bearer {bearer_token}"}
    return headers

def connect_to_endpoint(url, headers, params):
    response = requests.get(url, headers=headers, params=params)
    print(response.status_code)
    if response.status_code != 200:
        raise Exception(f"Request returned an error: {response.status_code} {response.text}")
    return response.json()

def analyze_sentiment(tweet):
    headers = {
        "Authorization": f"Bearer {API_KEY}",
        "Content-Type": "application/json"
    }
    data = {
        "model": "gpt-3.5-turbo",
        "messages": [{"role": "user", "content": f"Bu tweet Ankara Büyükşehir Belediyesine gönderilen bir tweettir. \
        Bu Tweeti içinde geçen kelimelere göre hangi hizmetle ilgiliyse 'Hizmet Türü: ' alanı altında sınıflandır. Tweetin içerisinde parsel veya imar keimleri geçiyorsa 'imar hizmetleri', \
        otobüs, durak, metro, ankaray, ulaşım, ego, trafik, yaya, yolcu, sefer, tuz, küreme, ring, bilet, dolmuş, minibüs, otopark, kelimelerinden biri veya birden fazlası geçiyorsa 'ulaşım hizmeti',\
        cadde, sokak, kaldırım, yol, kasis, kavşak, meydan, çukur, asfalt, aydınlatma, yama kelimelerinden biri veya birden fazlası geçiyorsa 'yol hizmetleri', \
        zabıta, ruhsat, denetim, ceza kelimelerinden biri veya birden fazlası geçiyorsa 'denetim hizmetleri', \
        inşaat, hafriyat, harfiyat moloz kelimelerinden bir veya birden fazlası geçiyorsa 'hafriyat hizmetleri', \
        ağaç, park, konser, engelli, yaşlı, aile, tesis, lokal, ekmek, çoçuk, spor, sporcu, çiçek, hayvan, köpek, kedi, belmek, müze, fomget, kadın kelimelerinden bir veya birden fazlası geçiyorsa 'sosyal hizmetler', \
        aski, su, sulama, rögar, lağım, altyapı, sel, yağmur, sayaç kelimelerinden bir veya birden fazlası geçiyorsa su ve 'kanalizasyon hizmetleri', \
        mezar cenaze, kelimelerinden bir veya birden fazlası geçiyorsa 'cenaze hizmetleri', tarım, gübre, kırsal, çiftçi kelimelerinden bir veya birden fazlası geçiyorsa 'kırsal kalkınma hizmetleri', \
        olarak sınıflandır, örneğin ulaşım hizmetleri, örneğin imar hizmetleri gibi. \
        Bu kelimelerden hiç biri geçmiyorsa 'ilgisiz' olarak sınıflandır. Daha sonra Tweeti duygu durumuna göre olumlu, olumsuz ve nötr olarak analiz et ve farklı bir alanda kaydet. : '{tweet}'"}]
    }
    response = requests.post(API_URL, headers=headers, json=data)
    if response.status_code != 200:
        raise Exception(f"OpenAI API request returned an error: {response.status_code} {response.text}")
    return response.json()

def parse_sentiment_analysis(sentiment_analysis):
    lines = sentiment_analysis.split('\n')
    hizmet_sinifi = lines[0].split(': ')[1].strip() if len(lines) > 0 else ""
    duygu_durumu = lines[1].split(': ')[1].strip() if len(lines) > 1 else ""
    return hizmet_sinifi, duygu_durumu

def fetch_and_analyze_tweets(query, start_time, end_time, max_results):
    headers = create_headers(BEARER_TOKEN)
    query += " -is:retweet"  # Retweetleri hariç tutmak için sorguya ekleme yapılıyor
    params = {
        'query': query,
        'start_time': start_time,
        'end_time': end_time,
        'max_results': max_results,
        'tweet.fields': 'created_at,text'
    }
    json_response = connect_to_endpoint(search_url, headers, params)
    tweets = json_response.get('data', [])

    result = []
    for tweet in tweets:
        try:
            sentiment_response = analyze_sentiment(tweet['text'])
            print("OpenAI API response:", sentiment_response)  # API yanıtını logla
            if 'choices' in sentiment_response and len(sentiment_response['choices']) > 0:
                sentiment_analysis = sentiment_response['choices'][0]['message']['content'].strip()
                hizmet_sinifi, duygu_durumu = parse_sentiment_analysis(sentiment_analysis)
                result.append({
                    'tweet_created_at': tweet['created_at'],
                    'id_str': tweet['id'],
                    'text': tweet['text'],
                    'hizmet_sinifi': hizmet_sinifi,
                    'duygu_durumu': duygu_durumu
                })
            else:
                print(f"Unexpected OpenAI API response format: {sentiment_response}")
        except IndexError as e:
            print(f"Index error: {e}")
        except Exception as e:
            print(f"An error occurred: {e}")
    return result

def save_tweets_to_json(tweets, filename):
    with open(filename, 'w', encoding='utf-8') as f:
        json.dump(tweets, f, ensure_ascii=False, indent=4)

def save_tweets_to_excel(tweets, filename):
    df = pd.DataFrame(tweets)
    df.to_excel(filename, index=False)

def on_submit():
    query = query_entry.get().strip()
    if not query:
        messagebox.showerror("Arama Terimi", "Bu alan boş olamaz.")
        return
    
    start_date = f"{start_year_cb.get()}-{start_month_cb.get()}-{start_day_cb.get()}T{start_hour_cb.get()}:{start_minute_cb.get()}:00Z"
    end_date = f"{end_year_cb.get()}-{end_month_cb.get()}-{end_day_cb.get()}T{end_hour_cb.get()}:{end_minute_cb.get()}:00Z"
    
    try:
        max_results = int(max_results_entry.get())
    except ValueError:
        messagebox.showerror("Geçersiz Değer", "Lütfen en fazla değeri giriniz")
        return

    try:
        tweets = fetch_and_analyze_tweets(query, start_date, end_date, max_results)
        # tweets değişkenini global yaparak diğer fonksiyonlarda kullanılmasını sağlayın
        global fetched_tweets
        fetched_tweets = tweets
        if not fetched_tweets:
            messagebox.showinfo("Sonuç Yok", "Belirtilen kriterlere uygun tweet bulunamadı.")
            return
    except Exception as e:
        messagebox.showerror("Hata", str(e))
        return
    
    messagebox.showinfo("Başarılı", "Tweetler başarıyla analiz edildi")

def save_to_json():
    if not fetched_tweets:
        messagebox.showinfo("Bilgi", "Kaydedilecek analiz bulunamadı.")
        return
    json_filename = filedialog.asksaveasfilename(defaultextension=".json", filetypes=[("JSON Dosyası", "*.json")])
    if json_filename:
        save_tweets_to_json(fetched_tweets, json_filename)
        messagebox.showinfo("Başarılı", f"Tweetler kaydedildi {json_filename}")

def save_to_excel():
    if not fetched_tweets:
        messagebox.showinfo("Bilgi", "Kaydedilecek analiz bulunamadı.")
        return
    excel_filename = filedialog.asksaveasfilename(defaultextension=".xlsx", filetypes=[("Excel Dosyası", "*.xlsx")])
    if excel_filename:
        save_tweets_to_excel(fetched_tweets, excel_filename)
        messagebox.showinfo("Başarılı", f"Tweetler kaydedildi {excel_filename}")

app = tk.Tk()
app.title("Twitter Duygu Analizi TurboGPT")
app.geometry("950x610")
app.resizable(False, False)

base_path = os.getcwd()
logo_path = os.path.join(os.path.dirname(os.path.realpath(sys.executable)), 'gazilogo.png')

# Gazi Üniversitesi logosunu yüklemek için yol güncellendi
if getattr(sys, 'frozen', False):
    logo_path = os.path.join(sys._MEIPASS, 'gazilogo.png')
else:
    logo_path = 'gazilogo.png'

# Resmi yükleme ve tkinter arayüz elemanları
try:
    image = Image.open(logo_path)
    image = image.resize((250, 250), Image.LANCZOS)
    logo = ImageTk.PhotoImage(image)
    logo_label = tk.Label(app, image=logo)
    logo_label.place(relx=1.0, rely=0, anchor='ne')
except Exception as e:
    messagebox.showerror("Logo Yükleme Hatası", f"Logo yüklenemedi: {e}")

tk.Label(app, text="Arama Kelimesini Girin:", font=("Helvetica", 14)).grid(row=0, column=0, columnspan=6, padx=10, pady=10, sticky="w")
query_entry = tk.Entry(app, width=40, font=("Helvetica", 12))
query_entry.grid(row=1, column=0, columnspan=6, padx=10, pady=10, sticky="w")

# Combobox Değeri
years = [str(year) for year in range(2024, 2025)]
months = [f"{month:02}" for month in range(1, 13)]
days = [f"{day:02}" for day in range(1, 32)]
hours = [f"{hour:02}" for hour in range(0, 24)]
minutes = [f"{minute:02}" for minute in range(0, 60)]

# Varsayılan tarih ve saatler (şu anki tarih ve bir saat sonrası)
now = datetime.utcnow()
default_start_time = now 
default_end_time = now

# Başlangıç Tarihi
tk.Label(app, text="Başlangıç Tarihi:", font=("Helvetica", 14)).grid(row=2, column=0, padx=10, pady=10, sticky="w")

start_frame = tk.Frame(app)
start_frame.grid(row=3, column=0, padx=10, pady=5, sticky="w")

start_year_cb = ttk.Combobox(start_frame, values=years, width=5, font=("Helvetica", 12))
start_year_cb.set(default_start_time.year)
start_year_cb.grid(row=0, column=0, padx=(0, 5), pady=5, sticky="w")
start_month_cb = ttk.Combobox(start_frame, values=months, width=3, font=("Helvetica", 12))
start_month_cb.set(f"{default_start_time.month:02}")
start_month_cb.grid(row=0, column=1, padx=(0, 5), pady=5, sticky="w")
start_day_cb = ttk.Combobox(start_frame, values=days, width=3, font=("Helvetica", 12))
start_day_cb.set(f"{default_start_time.day:02}")
start_day_cb.grid(row=0, column=2, padx=(0, 5), pady=5, sticky="w")
start_hour_cb = ttk.Combobox(start_frame, values=hours, width=3, font=("Helvetica", 12))
start_hour_cb.set(f"{default_start_time.hour:02}")
start_hour_cb.grid(row=0, column=3, padx=(0, 5), pady=5, sticky="w")
start_minute_cb = ttk.Combobox(start_frame, values=minutes, width=3, font=("Helvetica", 12))
start_minute_cb.set(f"{default_start_time.minute:02}")
start_minute_cb.grid(row=0, column=4, padx=(0, 5), pady=5, sticky="w")

# Bitiş Tarihi
tk.Label(app, text="Bitiş Tarihi:", font=("Helvetica", 14)).grid(row=4, column=0, padx=10, pady=10, sticky="w")

end_frame = tk.Frame(app)
end_frame.grid(row=5, column=0, padx=10, pady=5, sticky="w")

end_year_cb = ttk.Combobox(end_frame, values=years, width=5, font=("Helvetica", 12))
end_year_cb.set(default_end_time.year)
end_year_cb.grid(row=0, column=0, padx=(0, 5), pady=5, sticky="w")
end_month_cb = ttk.Combobox(end_frame, values=months, width=3, font=("Helvetica", 12))
end_month_cb.set(f"{default_end_time.month:02}")
end_month_cb.grid(row=0, column=1, padx=(0, 5), pady=5, sticky="w")
end_day_cb = ttk.Combobox(end_frame, values=days, width=3, font=("Helvetica", 12))
end_day_cb.set(f"{default_end_time.day:02}")
end_day_cb.grid(row=0, column=2, padx=(0, 5), pady=5, sticky="w")
end_hour_cb = ttk.Combobox(end_frame, values=hours, width=3, font=("Helvetica", 12))
end_hour_cb.set(f"{default_end_time.hour:02}")
end_hour_cb.grid(row=0, column=3, padx=(0, 5), pady=5, sticky="w")
end_minute_cb = ttk.Combobox(end_frame, values=minutes, width=3, font=("Helvetica", 12))
end_minute_cb.set(f"{default_end_time.minute:02}")
end_minute_cb.grid(row=0, column=4, padx=(0, 5), pady=5, sticky="w")

tk.Label(app, text="Kaç tweet getirilecek '10-100':", font=("Helvetica", 14)).grid(row=6, column=0, padx=10, pady=10, sticky="w")
max_results_entry = tk.Entry(app, width=10, font=("Helvetica", 12))
max_results_entry.grid(row=7, column=0, padx=10, pady=10, sticky="w")

tk.Button(app, text="Gönder", command=on_submit, font=("Helvetica", 14)).grid(row=8, column=0, padx=10, pady=10, sticky="w")

# Save buttons
save_json_button = tk.Button(app, text="Analizi JSON olarak Kaydet", command=save_to_json, font=("Helvetica", 14))
save_json_button.grid(row=10, column=0, padx=10, pady=10, sticky="w")

save_excel_button = tk.Button(app, text="Analizi Excel olarak Kaydet", command=save_to_excel, font=("Helvetica", 14))
save_excel_button.grid(row=11, column=0, padx=10, pady=10, sticky="w")

app.mainloop()


200
OpenAI API response: {'id': 'chatcmpl-9XE1DJXvGqj6TOYuDR6ko6CTqSk0p', 'object': 'chat.completion', 'created': 1717705739, 'model': 'gpt-3.5-turbo-0125', 'choices': [{'index': 0, 'message': {'role': 'assistant', 'content': 'Hizmet Türü: altyapı hizmetleri\nDuygu durumu: olumsuz'}, 'logprobs': None, 'finish_reason': 'stop'}], 'usage': {'prompt_tokens': 688, 'completion_tokens': 24, 'total_tokens': 712}, 'system_fingerprint': None}
OpenAI API response: {'id': 'chatcmpl-9XE1FZlDZJagTKgTTyq1LUxeYTFLJ', 'object': 'chat.completion', 'created': 1717705741, 'model': 'gpt-3.5-turbo-0125', 'choices': [{'index': 0, 'message': {'role': 'assistant', 'content': 'Hizmet Türü: ulaşım hizmeti\nDuygu Durumu: olumsuz'}, 'logprobs': None, 'finish_reason': 'stop'}], 'usage': {'prompt_tokens': 685, 'completion_tokens': 24, 'total_tokens': 709}, 'system_fingerprint': None}
OpenAI API response: {'id': 'chatcmpl-9XE1GaSWDc9RntYC6hkjd8KS4rj8y', 'object': 'chat.completion', 'created': 1717705742, 'model': 'gp

In [11]:
import tiktoken

metin = "sular kesik, perişan olduk. yardımcı olun lütfen. kelime arttıkça token artar"

tokenizer = tiktoken.get_encoding("cl100k_base")

tokenlar = tokenizer.encode(metin)

token_sayisi = len(tokenlar)

print(f"Token sayısı: {token_sayisi}")



Token sayısı: 31


In [26]:
pip install tiktoken

Defaulting to user installation because normal site-packages is not writeable
   ---------------------------------------- 0.0/799.0 kB ? eta -:--:--
   - ------------------------------------- 30.7/799.0 kB 435.7 kB/s eta 0:00:02
   --- ----------------------------------- 71.7/799.0 kB 787.7 kB/s eta 0:00:01
   ------- -------------------------------- 143.4/799.0 kB 1.2 MB/s eta 0:00:01
   ----------- ---------------------------- 235.5/799.0 kB 1.4 MB/s eta 0:00:01
   ---------------- ----------------------- 337.9/799.0 kB 1.6 MB/s eta 0:00:01
   ---------------------- ----------------- 450.6/799.0 kB 1.8 MB/s eta 0:00:01
   --------------------------- ------------ 553.0/799.0 kB 1.8 MB/s eta 0:00:01
   ------------------------------ --------- 614.4/799.0 kB 1.8 MB/s eta 0:00:01
   ---------------------------------- ----- 696.3/799.0 kB 1.8 MB/s eta 0:00:01
   ------------------------------------- -- 747.5/799.0 kB 1.7 MB/s eta 0:00:01
   ---------------------------------------- 799.0/7